### Necessary Imports

In [ ]:
import os
from openai import OpenAI
import chromadb
from dotenv import load_dotenv

#### Load OpenAI API and Initialize ChromaDB

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

chroma_client = chromadb.PersistentClient(path="chromadb/mobile_advisor")
plan_prices_collection = chroma_client.get_collection("plan_prices")
brand_migrations_collection = chroma_client.get_collection("brand_migration_deals")

### Retrieval Function

Define helper functions to query ChromaDB for relevant plans and generate a response using OpenAI.

In [14]:
def retrieve_relevant_docs(collection, query, n_results=15):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )

    docs = results["documents"][0]
    metadatas = results["metadatas"][0]

    return list(zip(docs, metadatas))

### RAG Function

The RAG function expands the query, retrieves matching entries, and uses GPT-4o-mini to pick the cheapest valid plan based on strict provider and brand migration rules.

In [ ]:
def rag_answer(query, collections):
    def expand_query(query):
        expansion = (
            " The customer might benefit from switching providers or taking brand migration deals."
        )
        return query + expansion
    
    query = expand_query(query)
    
    all_docs = []
    for col in collections:
        all_docs.extend(retrieve_relevant_docs(col, query))
    
    context = "\n\n".join([f"{doc}" for doc, _ in all_docs])

    prompt = f"""
    You are a strict mobile plan advisor. 
    1. NEVER recommend plans from the customer's current provider. 
    2. Check "offer prices" to make suggestions.
    3. Brand migrations may be recommended ONLY IF:
        a. Rogers owns Fido and Chatr. Migration only possible within themselves.
        b. Bell owns Virgin Plus and Lucky Mobile. Migration only possible within themselves.
        c. Telus owns Koodo Prepaid and Koodo Postpaid. Migration only possible within themselves.
        d. If ustomer is with Lucky Mobile, recommend Virgin Plus if no other cheapest option is available. 
        e. If customer is with Virgin Plus, recommend Bell if no other cheapest option is available 
        f. If customer is with Chatr, recommend Fido if no other cheapest option is available. 
        g. If customer is with Fido, recommend Rogers if no other cheapest option is available.
        h. New customers or customers who do not have an existing provider are not eligible for brand migration deals.
    4. Look for switching providers.
    5. Respond clearly with the provider name, plan name, and price. Bulletpoints, concise, factual, and to the point.
    6. If there isn't any good offer, say there is nothing available.
    7. Choose the cheapest deal that satisfies all contraints.

    Context:
    {context}

    Question: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that helps the advisor suggest good deals to the customer. Show key information in bulletpoints."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()


### Example Queries

In [16]:
query = "Customer is with Virgin Plus paying $50 per month. Are there any cheaper deals?"
print(rag_answer(query, [plan_prices_collection, brand_migrations_collection]))

Here are the best options available for the customer currently with Virgin Plus:

### Recommended Plan:
- **Provider:** Bell
- **Plan Name:** Migration Special - From Virgin Plus
- **Price:** $45 per month
- **Data:** 120 GB high-speed data on a 5G+ network (unlimited reduced-speed after limit)
- **Calling:** Unlimited within Canada and to U.S. + 1,000 international minutes to 27 countries
- **Texting:** Unlimited for Canada, U.S., and internationally
- **Streaming:** Supports HD video, includes access to Crave Basic
- **Additional Perks:** Waived activation fee, hotspot capabilities

This deal satisfies all constraints and is the cheapest option available for the customer.


In [17]:
query = "Customer is with Rogers paying $70 for 50GB. What is the cheapest plan available without data?"
print(rag_answer(query, [plan_prices_collection, brand_migrations_collection]))

- **Provider**: Fido  
  - **Plan Name**: 3GB Basic Plan - BYOP  
  - **Price**: $39.00 per month  
  - **Data**: 3 GB of high-speed data at 3 Mbps  
  - **Calling**: Unlimited calling within Canada and to the U.S.  
  - **International Calling**: 1000 international minutes to 27 countries  
  - **Texting**: Unlimited within Canada, limited to 1000 international texts  
  - **HD Streaming**: Not supported  
  - **Additional Perks**: $15 charge for every 100 MB of data overage.  

This is the cheapest option available while switching providers and considering brand migration constraints.


In [18]:
query = "Customer wants a new connection with maximum possible data at the highest possible speed. Price is not an issue."
print(rag_answer(query, [plan_prices_collection, brand_migrations_collection]))

Based on your requirements for maximum data at the highest speeds, here’s the best option available:

### Recommendation:
- **Provider:** Telus 
- **Plan Name:** 5G+ Complete Explore
- **Price:** $110 per month (discounted from $120)
    
### Key Features:
- **Data:** 250 GB of high-speed 5G+ data, reduced to 256 Kbps after reaching the limit
- **Calling:** Unlimited calls within Canada and the U.S., unlimited international minutes to 68 countries
- **Texting:** Unlimited texting within Canada, the U.S., and internationally
- **Hotspot Capability:** Yes, with data sharing
- **Streaming:** Supports HD video streaming
- **Additional Perks:** 
  - Premium financing options for device discounts
  - 5-Year Rate Plan Price Lock
  - Easy Roam starting at $5 per day

This plan offers the highest data cap and speeds while including comprehensive calling, texting, and additional perks.


In [19]:
query = "Customer is travelling to US. Which plan can I recommend?"
print(rag_answer(query, [plan_prices_collection, brand_migrations_collection]))

Based on the information available and the customer's need for calling in the U.S., I recommend the following plan:

- **Provider**: Bell
- **Plan Name**: Max - CAN/U.S.
- **Price**: $65 per month (discounted)
- **Key Features**:
  - 175 GB of high-speed data with 5G+ connectivity
  - Unlimited calling within Canada and the U.S.
  - 1000 international minutes to 27 countries
  - Roaming available in Canada and the U.S.
  - Unlimited international texting
  - HD video streaming
  - Access to services like Crave Basic and Perplexity Pro AI for one year

This plan provides comprehensive coverage for U.S. calls and includes substantial data and unlimited texting, making it suitable for the customer’s travel needs.


In [22]:
query = "Customer is currently with Chatr paying $25 per month for talk and text only. Is there any plan cheaper than $25?"
print(rag_answer(query, [plan_prices_collection, brand_migrations_collection]))

- **Recommendation**: Nothing available.  
There are no mobile plans cheaper than $25 that meet the specified criteria.
